In [1]:
!pip install stocksera
!pip install yfinance

In [2]:
import stocksera
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [3]:
discover = stocksera.Discover()

df = discover.jim_cramer()
df

In [4]:
count_df = df.groupby("Ticker").agg({"Call": "count"}).sort_values(by="Call", ascending=False)
count_df.reset_index(inplace=True)
count_df.head(10)

In [5]:
def get_stock_data(ticker_selected):
    ticker_df = df[df["Ticker"] == ticker_selected]
    
    history_df = yf.Ticker(ticker_selected).history(period="1y", interval="1d")
    history_df.reset_index(inplace=True)
    history_df["Date"] = history_df["Date"].astype(str)

    latest_price = history_df.iloc[-1]["Close"]
    ticker_df = ticker_df.merge(history_df, on="Date")
    ticker_df["Pro Cramer"] = latest_price
    ticker_df["Pro Cramer"] = 100 * (ticker_df["Pro Cramer"] - ticker_df["Close"]) / ticker_df["Close"]
    ticker_df.loc[ticker_df["Call"].isin(["Negative", "Sell"]), 'Pro Cramer'] *= -1
    avg_return = round(ticker_df["Pro Cramer"].mean(), 2)
    return ticker_df, history_df, avg_return

In [6]:
palette ={"Buy": "green", "Positive": "yellow", "Hold": "grey", "Negative": "orange", "Sell": "red"}

In [9]:
list_of_tickers = count_df["Ticker"].head(40).tolist()
fig, axes = plt.subplots(10, 4, sharex=True, figsize=(35,45))
fig.suptitle(f"Jim Cramer Stocks Recommendations Performance Compiled by Stocksera as of {str(datetime.utcnow().date())}", fontsize=30)

row_num = 0
col_num = 0
for index, ticker in enumerate(list_of_tickers):
    ticker_df, history_df, avg_return = get_stock_data(ticker)
    
    axes[row_num, col_num].set_title(f"{ticker} ({avg_return}%)", fontsize=15, weight="bold")
    sns.lineplot(ax=axes[row_num, col_num], data=history_df, x="Date", y="Close", color="black", linewidth=0.5, alpha=0.7)
    sns.scatterplot(ax=axes[row_num, col_num], data=ticker_df, x="Date", y="Close", hue="Call", palette=palette)  # style="Call"
    
    for dir in ["left", "right", "top", "bottom"]:
        axes[row_num, col_num].spines[dir].set_linewidth(2)
        axes[row_num, col_num].spines[dir].set_color("red" if avg_return < 0 else "green")
        axes[row_num, col_num].legend(title="")
        # axes[row_num, col_num].get_legend().remove()
    
    col_num += 1
    if col_num == 4:
        row_num += 1
        col_num = 0
        
plt.xticks(range(0, len(history_df), 50))
fig.tight_layout(rect=[0, 0.03, 1, 0.97])  
fig.show()
plt.savefig("jim_cramer.png", facecolor="white", transparent=False)